In [1]:
import deepcut
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras
from pythainlp import word_vector
import re,string
from numpy import argmax
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import random

In [2]:
def clean_msg(msg):
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<*?>','', msg)
    msg = re.sub(r'\d+', '',msg)
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    msg = re.sub(r'-','',msg)
    msg = re.sub(r'ฯ','',msg)
    msg = re.sub(r'ๆ','',msg)
    msg = re.sub(r'!@#$','',msg)
    msg = re.sub(r'[a-zA-Z]','',msg)
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

In [ ]:
input_txt = open("corpus1.txt","r")
text = input_txt.read()
txt = clean_msg(text)
input_txt.close()
corpus = txt.split()
print(corpus)

In [ ]:
def word_tokenize(corpus):
    x = 0
    words = []
    for i in corpus:
        list_word = deepcut.tokenize(i,custom_dict='custom_dict.txt') #ตัดคำ
        words.append(list_word)
        x += 1
    return words
# corpus=word_tokenize(corpus)
# x_test = word_tokenize(x_test)
words = word_tokenize(corpus)
print(words)
print(len(words))

In [ ]:
def ran_incorect(w):
    incorect = []
    for i in w:
        in_cor = []
        for r in i:
            if len(r) >= 4:
                x = random.randint(1, len(r)-1)
                y = random.randint(0, len(thai_letters)-1)
                x1 = []
                for ran in r:
                    x1.append(ran)
                x1[x] = thai_letters[y]
                j = ''.join(x1)
                in_cor.append(j)
            else:
                in_cor.append(r)
        incorect.append(in_cor)
    return incorect
#                 print(r[1])
#             print(len(r))

In [ ]:
thai_letters = 'กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรฤฤๅลฦฦๅวศษสหฬอฮะัาำิีึืุูเแโใไ็่้๊๋์'

In [ ]:
random.seed(1)
incorect_word = ran_incorect(words)
print(incorect_word)
print(len(incorect_word))

In [ ]:
def create_tar(word,ch):
    tar_in = []
    for i in word:
        targ = []
        for j in i:
            if ch:
                if len(j) >= 5:
                    targ.append('T')
                else:
                    targ.append('F')
            else:
                targ.append('T')
        tar_in.append(targ)
    return tar_in
# print(tar_in)

In [ ]:
target_c = create_tar(words,False)
target_ic = create_tar(incorect_word,True)

In [ ]:
target_c.extend(target_ic)
words.extend(incorect_word)

In [ ]:
print(words)

In [ ]:
char_to_int = dict((c, i) for i, c in enumerate(thai_letters))
int_to_char = dict((i, c) for i, c in enumerate(thai_letters))

In [ ]:
x_tr = []
for i in words:
    x_tr.extend(i)
print(len(x_tr))

In [ ]:
y_tr = []
for i in target_c:
    y_tr.extend(i)
print(len(y_tr))

In [ ]:
x_tr_ch = []
for char1 in x_tr:
    integer_encoded = [char_to_int[char] for char in char1]
    x_tr_ch.append(integer_encoded)
print(len(x_tr_ch))

In [ ]:
check_idx = {}
# check_idx['pad'] = 0
check_idx['T'] = 0
check_idx['F'] = 1

In [ ]:
y_train = []
for i in y_tr:
    if i == 'T':
        y_train.append(1)
    else:
        y_train.append(0)
print(len(y_train))
print(y_train)

In [18]:
max_len = 30
x_train = sequence.pad_sequences(np.array(x_tr_ch),maxlen=max_len,padding='pre')
print(x_train.shape)

(1614, 30)


In [36]:
model = keras.Sequential()
model.add(keras.layers.Embedding(len(thai_letters)+1,50,input_length=max_len))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(32, return_sequences=True)))
model.add(keras.layers.Bidirectional(keras.layers.LSTM(12)))
model.add(keras.layers.Dense(64, activation=keras.activations.relu))
model.add(keras.layers.Dense(2))

In [37]:
model.compile(loss=keras.losses.sparse_categorical_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=1e-3),metrics=['accuracy'])

In [38]:
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 50)            3600      
_________________________________________________________________
bidirectional_4 (Bidirection (None, 30, 64)            21248     
_________________________________________________________________
bidirectional_5 (Bidirection (None, 24)                7392      
_________________________________________________________________
dense_18 (Dense)             (None, 64)                1600      
_________________________________________________________________
dense_19 (Dense)             (None, 2)                 130       
Total params: 33,970
Trainable params: 33,970
Non-trainable params: 0
_________________________________________________________________
None


In [39]:
model.fit(x_train, np.array(y_train), epochs=10,validation_split=0.1)

Epoch 1/10
46/46 [==============================] - 2s 39ms/step - loss: 1.2200 - accuracy: 0.5840 - val_loss: 0.6931 - val_accuracy: 0.7531
Epoch 2/10
46/46 [==============================] - 0s 9ms/step - loss: 0.6931 - accuracy: 0.3230 - val_loss: 0.6931 - val_accuracy: 0.7284
Epoch 3/10
46/46 [==============================] - 0s 9ms/step - loss: 0.6931 - accuracy: 0.3189 - val_loss: 0.6931 - val_accuracy: 0.7284
Epoch 4/10
46/46 [==============================] - 0s 9ms/step - loss: 0.6931 - accuracy: 0.3189 - val_loss: 0.6931 - val_accuracy: 0.7284
Epoch 5/10
46/46 [==============================] - 0s 9ms/step - loss: 0.6931 - accuracy: 0.3189 - val_loss: 0.6931 - val_accuracy: 0.7284
Epoch 6/10
46/46 [==============================] - 0s 9ms/step - loss: 0.6931 - accuracy: 0.3189 - val_loss: 0.6931 - val_accuracy: 0.7284
Epoch 7/10
46/46 [==============================] - 0s 10ms/step - loss: 0.6931 - accuracy: 0.3189 - val_loss: 0.6931 - val_accuracy: 0.7284
Epoch 8/10
46/46 [

In [40]:
x = model.predict(x_train).argmax(axis=1)
print(len(x))

1614


In [43]:
for i in x:
    if i == 1:
        print(i)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
